In [1]:
! pip install kfp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 17.4 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.11.0-py3-none-any.whl size=366403 sha256=5aff2972c33182ad111ea21e14eaae271d0da4587e53a24ff25a6ab03e06c9c4
  Stored in directory: /root/.cache/pip/wheels/a2/20/d2/b43519f79db0c7d6530f59dc4aabd9f450bccf5e018aee59aa
  Created wheel for kfp-server-api: filename=kfp_server_api-2.3.0-py3-none-any.whl size=116407 sha256=b68006dc91f71c78d481937e711b472e13bd25df539ef1945afc09f3662bb952
  Stored in directo

In [2]:
!pip install google-cloud-pipeline-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 46.5 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.10.1-py3-none-any.whl size=364799 sha256=49f72c8f066e63e073054bf8df0acd638934ba89a093a50c95155b9197725ff3
  Stored in directory: /root/.cache/pip/wheels/ec/3a/e6/68b6a5de8fec76d3e68c2ca3c7149c81aff126da47c105417d
Successfully built kfp
  Attempting uninstall: kfp-pipeline-spec
    Found existing installation: kfp-pipeline-spec 0.6.0
    Uninstalling kfp-pipeline-spec-0.6.0:
      Successfully uninstalled kfp-pipeline-spec-0.6.0
  Attempting uninstall: kfp
    Found existing installation: kfp 2.11.0
    Uninstalling kfp-2.11.0:
      Successfully uninstalled kfp-2.11.0


In [3]:
!pip install gcsfs

In [4]:
!pip install fsspec

In [5]:
pip install scikit-learn

In [6]:
project_id = 'gcp-final-project-444704'
location = 'us-central1'

In [7]:
from google.cloud import aiplatform
aiplatform.init(project=project_id, location=location)

from kfp.v2.dsl import pipeline, component, InputPath, OutputPath, Dataset
import joblib
import gcsfs
import fsspec
import pandas as pd
import numpy as np

<ipython-input-7-134b2d31f0ea>:4: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from kfp.v2.dsl import pipeline, component, InputPath, OutputPath, Dataset


In [8]:
 from kfp.v2.dsl import pipeline, component, InputPath, OutputPath

# Step 1: Initial Data Preparation (Reuse the same component as in training)
@component(packages_to_install=["pandas", "numpy", "fsspec", "gcsfs"])
def initial_data_preparation(
    input_dataset_path: str,  # Input: Raw dataset path
    prepared_dataset_path: OutputPath("Dataset")  # Output: Cleaned and prepared dataset path
):
    import pandas as pd
    import numpy as np

    # Load dataset
    df = pd.read_csv(input_dataset_path)

    # Handle missing values
    df['Exercise Frequency'].fillna('None', inplace=True)
    df['Type of Treatment'].fillna('None', inplace=True)
    df['Number of Prior Visits'].fillna(df['Number of Prior Visits'].median(), inplace=True)
    df['Medications Prescribed'].fillna(df['Medications Prescribed'].mean(), inplace=True)

    # Handle outliers
    df['Age'] = df['Age'].clip(upper=df['Age'].quantile(0.99))
    df['Adjusted Weight (kg)'] = df['Adjusted Weight (kg)'].clip(upper=df['Adjusted Weight (kg)'].quantile(0.95))
    df['Length of Stay'] = np.log1p(df['Length of Stay'])
    df['Number of Prior Visits'] = df['Number of Prior Visits'].clip(upper=df['Number of Prior Visits'].quantile(0.95))

    # Drop unnecessary columns
    columns_to_drop = ['Hospital ID', 'Weight (kg)']
    df = df.drop(columns=columns_to_drop)

    # Save the prepared dataset
    df.to_csv(prepared_dataset_path, index=False)

# Step 2: Perform Validation (Normalize the test dataset using the scaler from training pipeline)
@component(packages_to_install=["pandas", "numpy", "joblib", "gcsfs", "scikit-learn"])
def perform_validation(
    validation_dataset_path: InputPath("Dataset"),  # Input: Raw test dataset
    scaler_path: str,                              # Input: Path to saved scaler
    normalized_validation_dataset_path: OutputPath("Dataset")  # Output: Processed test dataset
):
    import pandas as pd
    import joblib
    import gcsfs

    # Load validation dataset
    df_validate = pd.read_csv(validation_dataset_path)

    # Extract PatientID for later use
    if 'PatientID' in df_validate.columns:
        patient_ids = df_validate['PatientID']
    else:
        raise ValueError("PatientID column is missing in the input dataset.")

    # Drop PatientID temporarily for transformations
    df_validate = df_validate.drop(columns=['PatientID'])

    # Load the scaler
    fs = gcsfs.GCSFileSystem()
    with fs.open(scaler_path, 'rb') as f:
        scaler = joblib.load(f)

    # Categorical encoding: Binary encoding for 'Gender' and 'Smoker'
    df_validate['Gender'] = df_validate['Gender'].map({'Male': 0, 'Female': 1})
    df_validate['Smoker'] = df_validate['Smoker'].astype(int)

    # One-hot encoding for categorical variables
    categorical_columns = ['Ethnicity', 'Diet Type', 'Type of Treatment', 'Exercise Frequency']
    df_validate = pd.get_dummies(df_validate, columns=categorical_columns, drop_first=False)

    # Ensure numerical columns match the scaler's expected input
    numerical_columns = ['Height (m)', 'BMI', 'Adjusted Weight (kg)', 'Number of Prior Visits',
                         'Medications Prescribed', 'Length of Stay']

    # Apply scaling to numerical features
    df_validate[numerical_columns] = scaler.transform(df_validate[numerical_columns])

    # Match column order used in training pipeline
    #expected_columns = scaler.feature_names_in_  # Retrieve expected column order from the scaler
    #df_validate = df_validate.reindex(columns=expected_columns, fill_value=0)

    # Reattach the PatientID column at the start
    df_validate.insert(0, 'PatientID', patient_ids)

    # Save the normalized validation dataset
    df_validate.to_csv(normalized_validation_dataset_path, index=False)




@component(packages_to_install=["pandas", "joblib", "gcsfs", "scikit-learn", "xgboost"])
def perform_predictions(
    dataset_for_prediction_path: InputPath("Dataset"),
    model_path: str,
    predictions_path: OutputPath("Dataset"),
    gcs_dump_path: str  # Additional input: Path to dump the test data
):
    import pandas as pd
    import joblib
    import gcsfs

    # Load the dataset for predictions
    df = pd.read_csv(dataset_for_prediction_path)

    # Extract the PatientID column for output
    if 'PatientID' in df.columns:
        patient_ids = df['PatientID']
        features = df.drop(columns=['PatientID'])
    else:
        raise ValueError("PatientID column is missing in the input dataset.")

    # Align columns to match the model's training features
    fs = gcsfs.GCSFileSystem()
    with fs.open(model_path, 'rb') as f:
        model = joblib.load(f)
    expected_columns = model.feature_names_in_
    features = features.reindex(columns=expected_columns, fill_value=0)

    # Dump the testing data to GCS for review
    with fs.open(f"{gcs_dump_path}/xgb_test_data.csv", 'w') as f:
        features.to_csv(f, index=False)
    print("Testing DataFrame dumped to GCS.")

    # Print a preview of the data
    print("Testing DataFrame (features) Preview:")
    print(features.head())

    # Make predictions
    predictions = model.predict(features)

    # Save predictions
    output_df = pd.DataFrame({
        'PatientID': patient_ids,
        'readmission_prediction': predictions
    })
    output_df.to_csv(predictions_path, index=False)


# Step 4: Define the Inference Pipeline
@pipeline(name="healthcare-readmissions-inference-pipeline")
def healthcare_inference_pipeline(
    input_test_dataset_path: str,   # Input: Test dataset
    scaler_uri: str,                # Input: Scaler from training pipeline
    model_uri: str                  # Input: Model from training pipeline
):
    # Step 1: Prepare the test dataset
    prepared_test_data = initial_data_preparation(
        input_dataset_path=input_test_dataset_path
    )

    # Step 2: Normalize the test dataset
    normalized_test_data = perform_validation(
        validation_dataset_path=prepared_test_data.outputs["prepared_dataset_path"],
        scaler_path=scaler_uri
    )

    # Step 3: Perform predictions
    perform_predictions(
    dataset_for_prediction_path=normalized_test_data.outputs["normalized_validation_dataset_path"],
    model_path=model_uri,
    gcs_dump_path="gs://healthcare_readmissions/debugdatasets"  # Replace with your GCS bucket path
)


# Compile and Run the Pipeline
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=healthcare_inference_pipeline,
    package_path="healthcare_inference_pipeline.json"
)

# Run the pipeline
pipeline_job = aiplatform.PipelineJob(
    display_name="healthcare-readmissions-inference-pipeline",
    template_path="healthcare_inference_pipeline.json",
    pipeline_root="gs://healthcare_readmissions",
    parameter_values={
        "input_test_dataset_path": "gs://healthcare_readmissions/healthcare_readmissions_dataset_test.csv",
        "scaler_uri": "gs://healthcare_readmissions/853819008626/healthcare-readmissions-training-pipeline-20241216091749/split-and-preprocess-data_820267594517839872/scaler_path",  # Update with the actual scaler path from training
        "model_uri": "gs://healthcare_readmissions/853819008626/healthcare-readmissions-training-pipeline-20241216091749/train-model_-8403104442336935936/trained_model_path"  # Update with the actual model path from training
    },
    enable_caching=True
)

pipeline_job.run()





/usr/local/lib/python3.10/dist-packages/kfp/dsl/component_decorator.py:126: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.9' to 'python:3.10' on Oct 1, 2025. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.10.
  return component_factory.create_component_from_func(
INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/853819008626/locations/us-central1/pipelineJobs/healthcare-readmissions-inference-pipeline-20241218004423
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/853819008626/locations/us-central1/pipelineJobs/healthcare-readmissions-inference-pip